In [38]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import skimage.io
import cv2
from sklearn import model_selection

In [21]:
curr_dir = os.getcwd()
path = os.path.dirname(curr_dir)
print(path)
csv_file_path = os.path.join(path, 'DataCSV/multicam_full_h36m.csv')

/home/rasho/Falling-Person-Detection-based-On-AlphaPose/dataset


In [22]:
df = pd.read_csv(csv_file_path)

In [23]:
df.head()

,Unnamed: 0,image_id,idx,pos_class,box,score,Hip_x,Hip_y,RHip_x,RHip_y,...,LElbow_x,LElbow_y,LWrist_x,LWrist_y,RShoulder_x,RShoulder_y,RElbow_x,RElbow_y,RWrist_x,RWrist_y
0,0,01095.jpg,0.0,Falling,"[264.7388000488281, 97.85186004638672, 174.059...",2.473748,372.166229,201.207169,380.098633,201.207169,...,293.975372,130.948700,255.446533,140.014313,364.233826,133.215103,389.164246,128.682297,411.828278,103.751877
1,1,01096.jpg,0.0,Falling,"[270.8756103515625, 90.72824096679688, 167.931...",2.418112,365.774277,202.608543,373.427399,201.515244,...,279.403351,135.917068,254.257401,139.196991,353.747955,138.103683,377.800598,127.170654,395.293457,103.117996
2,2,01097.jpg,0.0,Falling,"[205.35069274902344, 93.36463165283203, 236.03...",2.625777,361.788422,201.726387,361.788422,203.263107,...,364.861847,123.353851,383.302460,101.839813,315.686920,135.647583,275.732300,141.794449,238.851105,147.941315
3,3,01098.jpg,0.0,Falling,"[207.93246459960938, 113.12434387207031, 227.2...",2.519368,354.134201,201.688332,355.613983,203.168106,...,352.654419,123.260284,367.452179,105.502991,326.018494,146.936676,346.735321,126.219833,367.452179,105.502991
4,4,01099.jpg,0.0,Falling,"[206.1958465576172, 104.33092498779297, 230.08...",2.612864,349.700714,212.788422,355.692566,211.290466,...,241.847382,148.376007,217.879974,157.363800,325.733307,145.380096,343.708862,118.416756,355.692566,100.441200


In [30]:
df.columns 


Index(['image_id', 'pos_class', 'box', 'score', 'Hip_x', 'Hip_y', 'RHip_x',
       'RHip_y', 'RKnee_x', 'RKnee_y', 'RFoot_x', 'RFoot_y', 'LHip_x',
       'LHip_y', 'LKnee_x', 'LKnee_y', 'LFoot_x', 'LFoot_y', 'Spine_x',
       'Spine_y', 'Thorax_x', 'Thorax_y', 'Nose_x', 'Nose_y', 'Head_x',
       'Head_y', 'LShoulder_x', 'LShoulder_y', 'LElbow_x', 'LElbow_y',
       'LWrist_x', 'LWrist_y', 'RShoulder_x', 'RShoulder_y', 'RElbow_x',
       'RElbow_y', 'RWrist_x', 'RWrist_y'],
      dtype='object')

In [24]:
df['pos_class'].value_counts()

Standing    37657
Falling     17286
Name: pos_class, dtype: int64

In [25]:
df = df.drop(['Unnamed: 0','idx'], axis=1)
df = df.sort_values(['score'],ascending=False)

In [26]:
df['pos_class'].value_counts()

Standing    37657
Falling     17286
Name: pos_class, dtype: int64

In [13]:
#df = df.drop_duplicates(subset = ['image_id','pos_class'], keep = 'first')
#df = df.sort_index()

In [28]:
csv_file_path = os.path.join(path, 'DataCSV/multicam_full_clean_h36m.csv')
df.to_csv(csv_file_path)

In [29]:
df.head()

,image_id,pos_class,box,score,Hip_x,Hip_y,RHip_x,RHip_y,RKnee_x,RKnee_y,...,LElbow_x,LElbow_y,LWrist_x,LWrist_y,RShoulder_x,RShoulder_y,RElbow_x,RElbow_y,RWrist_x,RWrist_y
49218,00652.jpg,Standing,"[364.3770446777344, 119.92179107666016, 129.16...",3.157302,446.041534,242.219315,432.377014,237.664474,416.435059,278.658051,...,459.706055,237.664474,423.267334,219.445099,436.931854,164.786987,430.099579,196.670883,414.157623,208.057983
3379,00746.jpg,Falling,"[270.775390625, 247.5062255859375, 261.8685302...",3.151284,405.119400,308.028717,403.414520,318.257965,457.970459,321.667694,...,372.726807,284.160492,393.185303,260.292267,331.809845,321.667694,362.497589,345.535919,382.956055,372.813904
2488,00378.jpg,Falling,"[306.2577819824219, 236.47711181640625, 237.99...",3.134247,446.947128,313.951828,442.298798,320.149597,476.386536,360.435089,...,395.815552,289.160736,374.123352,292.259644,383.420013,273.666321,355.530060,286.061859,321.442322,279.864105
5382,00362.jpg,Falling,"[185.53671264648438, 222.8995361328125, 259.96...",3.129288,293.518372,311.527176,279.978363,306.449677,246.128296,340.299713,...,384.913513,292.909637,418.763580,286.139648,310.443390,259.059601,293.518372,238.749557,303.673401,225.209534
5360,00340.jpg,Falling,"[182.25418090820312, 224.10716247558594, 262.6...",3.128088,294.790985,310.942612,281.108704,304.101471,243.482422,341.727753,...,383.725830,293.839752,417.931549,286.998627,311.893829,259.634033,294.790985,239.110626,305.052704,225.428345


In [47]:
def createKfolds(df):
    df2 = df
    df2["kfold"]=-1
    df2=df2.sample(frac=1).reset_index(drop=True)
    kf = model_selection.StratifiedKFold(n_splits=5,shuffle=True, random_state=42)
    
    for fold,(train_idx,val_idx) in enumerate(kf.split(X=df, y=df.pos_class.values)):
        print(len(train_idx), len(val_idx))
        df2.loc[val_idx,'kfold'] = fold
    
    return df2

In [48]:
df = createKfolds(df)

43954 10989
43954 10989
43954 10989
43955 10988
43955 10988


In [46]:
df[df.kfold==0]

,image_id,pos_class,box,score,Hip_x,Hip_y,RHip_x,RHip_y,RKnee_x,RKnee_y,...,LElbow_y,LWrist_x,LWrist_y,RShoulder_x,RShoulder_y,RElbow_x,RElbow_y,RWrist_x,RWrist_y,kfold
2488,00378.jpg,Falling,"[306.2577819824219, 236.47711181640625, 237.99...",3.134247,446.947128,313.951828,442.298798,320.149597,476.386536,360.435089,...,289.160736,374.123352,292.259644,383.420013,273.666321,355.530060,286.061859,321.442322,279.864105,0
5360,00340.jpg,Falling,"[182.25418090820312, 224.10716247558594, 262.6...",3.128088,294.790985,310.942612,281.108704,304.101471,243.482422,341.727753,...,293.839752,417.931549,286.998627,311.893829,259.634033,294.790985,239.110626,305.052704,225.428345,0
5364,00344.jpg,Falling,"[181.37062072753906, 224.11318969726562, 263.7...",3.121144,296.087387,310.281281,284.066162,303.412018,246.285202,341.192963,...,293.108093,418.016907,289.673462,311.543243,258.761749,294.370087,238.153961,304.673981,224.415421,0
5372,00352.jpg,Falling,"[184.49009704589844, 225.32041931152344, 259.6...",3.117467,295.731689,311.073746,282.206879,304.311340,245.013641,341.504578,...,294.167725,420.836212,287.405334,312.637695,256.974487,295.731689,236.687271,302.494110,223.162460,0
5392,00372.jpg,Falling,"[182.4438934326172, 224.13177490234375, 266.45...",3.116853,294.854996,312.688110,282.711823,307.483887,244.547546,342.178680,...,293.605988,418.021515,286.667023,310.467651,258.911194,293.120270,238.094315,303.528687,224.216385,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12973,00741.jpg,Falling,"[581.6762084960938, 88.1971206665039, 35.01599...",0.604403,601.436615,108.318649,593.428040,102.562492,609.445190,110.571060,...,144.857727,593.428040,142.605316,611.447327,109.069450,613.449463,110.070526,610.195984,144.857727,0
575,01176.jpg,Falling,"[667.8910522460938, 100.96480560302734, 42.292...",0.603424,681.285492,149.024910,680.137085,147.302322,680.137085,163.379776,...,140.986191,697.362915,148.450714,707.124268,159.934601,675.543518,131.799072,670.949951,137.541016,0
20385,00554.jpg,Standing,"[422.91162109375, 65.80513763427734, 37.691192...",0.601847,437.831039,101.154629,428.751770,90.357666,448.873383,77.106857,...,99.191544,421.390198,89.866898,435.622559,89.376129,430.224091,98.210007,420.899445,90.357666,0
15984,04303.jpg,Falling,"[4.989658832550049, 288.80712890625, 109.12468...",0.601343,60.262447,379.688431,61.683342,383.240662,70.208710,323.563110,...,344.876526,34.686348,360.506348,33.265450,374.715302,50.316185,415.921265,36.107239,360.506348,0
